In [129]:
def read_file(name):
    with open(name) as f:
        data = f.read()
    return data
        
documents_data = read_file('documents.txt')
keywords_data = read_file('keywords.txt')

def sum_squared(values):
    return reduce(lambda x, y: x + y, map(lambda x: x ** 2, list(values)))

In [85]:
# create documents
from nltk import word_tokenize
from nltk.stem import *

docs = documents_data.split('\n\n')
documents = []

for i, doc in enumerate(docs): 
    doc_data = doc.split('\n')
    title = doc_data[0]
    body = ' '.join(doc_data[1:len(doc_data)])

    words = word_tokenize(doc)
    words = [word for word in words if word.isalpha()]
    tokens = [stemmer.stem(word) for word in words]
                        
    documents.append({'idx': i, 'title': title, 'body': body, 'tokens': tokens})

In [86]:
# create keywords 
keywords = word_tokenize(keywords_data)
keywords = [word for word in keywords if word.isalpha()]
keywords = [stemmer.stem(word) for word in keywords]

keywords_vector = {}

for keyword in keywords: 
    keywords_vector[keyword]= 0
    

In [ ]:
# create bag of words
bag_of_words = []

for i, doc in enumerate(documents):
    keywords_vec = keywords_vector.copy()
    for token in doc['tokens']:
        if token in keywords:
            keywords_vec[token] += 1
            
    bag_of_words.append({'idx': i, 'vector': keywords_vec, 'max': max(keywords_vec.values())})
    
bag_of_words


In [130]:
# tf representation
from functools import reduce
from math import sqrt
tf = []

for bag in bag_of_words:
    idx = bag['idx']
    vec = bag['vector']
    max_n = bag['max']

    keywords_vec = keywords_vector.copy()
    
    for key in vec.keys():
        keywords_vec[key] = vec[key]/max_n
    
    vector_length = sqrt(sum_squared(keywords_vec.values()))
    
    tf.append({'idx': idx, 'vector': keywords_vec, 'vector_length': vector_length})
    

In [132]:
# idf representation
from math import log10

idf = keywords_vector.copy()

document_count = len(documents)

for key in idf.keys():
    word_count = 0
    
    for doc in documents:
        if key in doc['tokens']:
            word_count += 1
    
    idf[key] = word_count / document_count


In [135]:
# tfidf representation

tfidf = []

for i, doc in enumerate(tf):
    keywords_vec = keywords_vector.copy()

    for key in doc['vector'].keys():
        keywords_vec[key] = doc['vector'][key] * idf[key]
        
    vector_length = sqrt(sum_squared(keywords_vec.values()))
        
    tfidf.append({'idx': i, 'vector': keywords_vec, 'vector_length': vector_length})
    

88